QUESTION TO BE ANSWERED:
Do developing or developed nations produce more power per capita? - Carly

In [87]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

global_power_path = "Resources/global_power_plant_database.csv"
global_gni_path = "Resources/Global_gni.csv"
global_population_path = "Resources/global_population_dataset.csv"

global_powerdf = pd.read_csv(global_power_path)
global_gnidf = pd.read_csv(global_gni_path)
global_population = pd.read_csv(global_population_path)

#global_gnidf.head()

In [88]:
#global_powerdf.head()

In [89]:
global_population.head

global_population_cleandf = global_population[['Country Name', 'Country Code', '2020']]

#global_population_cleandf.head()

In [90]:
global_power_cleandf = global_powerdf[['country', 'country_long', 'name', 'capacity_mw', 'latitude', 'longitude', 'primary_fuel' ]]

global_power_cleandf.rename(columns={'country_long': 'Country Name',  'country': 'Country Code'}, inplace=True)

#global_power_cleandf.head()

In [91]:
global_gni_cleandf = global_gnidf[['Country Name', 'Country Code', '2020']]
global_gni_cleandf = global_gni_cleandf.dropna()
#global_gni_cleandf.head()

In [92]:
merged_power_plantdf = pd.merge(global_power_cleandf, global_gni_cleandf, how="left", on=["Country Code", "Country Code"])
merged_power_plantdf.head(50)

merged_power_plantdf =  merged_power_plantdf[['Country Code', 'Country Name_x', 'name', 'capacity_mw', 'latitude',
       'longitude', 'primary_fuel', '2020']]

#merged_power_plantdf.head()

In [93]:
merged_power_plantdf.rename(columns={'Country Name_x': 'Country Name',  '2020': 'GNI', 'capacity_mw': 'Power Plant Capacity(MWh)', 'latitude': 'Latitude',
'longitude': 'Longitude', 'name': 'Power Plant Name', 'primary_fuel': 'Primary Fuel'}, inplace=True)
#merged_power_plantdf.head()

In [94]:
merged_power_plantdf = pd.merge(merged_power_plantdf, global_population_cleandf, how="left", on=["Country Code", "Country Code"])
#merged_power_plantdf.head()

In [95]:
merged_power_plantdf.columns

merged_power_plantdf = merged_power_plantdf[['Country Code', 'Country Name_x', 'Power Plant Name',
       'Power Plant Capacity(MWh)', 'Latitude', 'Longitude', 'Primary Fuel', 'GNI', '2020']]

#merged_power_plantdf.head()

In [96]:
merged_power_plantdf.rename(columns={'Country Name_x': 'Country Name',  '2020': 'Population'}, inplace=True)

merged_power_plantdf.head()

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.322,65.1190,Hydro,500.0,38928341.0
1,AFG,Afghanistan,Kandahar DOG,10.0,31.670,65.7950,Solar,500.0,38928341.0
2,AFG,Afghanistan,Kandahar JOL,10.0,31.623,65.7920,Solar,500.0,38928341.0
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.556,69.4787,Hydro,500.0,38928341.0
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.641,69.7170,Hydro,500.0,38928341.0


In [97]:
#> 12615 - developing country
#< 12615 = developed country

merged_power_plantdf['GNI'].max()

bins = [0 , 12614, 83000]
group_Labels = ['Developing', 'Developed']

#pd.cut(merged_power_plantdf["GNI"], bins, labels=group_Labels).head()

In [98]:
merged_power_plantdf["Country Status"] = pd.cut(merged_power_plantdf["GNI"], bins, labels=group_Labels)
#merged_power_plantdf.head()

In [99]:
# Create new dataframe to work in.
power_plant_capacity_df = merged_power_plantdf
# power_plant_capacity_df

In [100]:
# Add a new column to calculate the MW capacity for a year 
# (so it's a more manageable number to work with when divided by the population)
# There are 8760 hours in a year.
power_plant_capacity_df["Power Plant Capacity(MW per Year)"] = ((power_plant_capacity_df["Power Plant Capacity(MWh)"]) * (8760))
power_plant_capacity_df

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population,Country Status,Power Plant Capacity(MW per Year)
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.3220,65.1190,Hydro,500.0,38928341.0,Developing,289080.0
1,AFG,Afghanistan,Kandahar DOG,10.0,31.6700,65.7950,Solar,500.0,38928341.0,Developing,87600.0
2,AFG,Afghanistan,Kandahar JOL,10.0,31.6230,65.7920,Solar,500.0,38928341.0,Developing,87600.0
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.5560,69.4787,Hydro,500.0,38928341.0,Developing,578160.0
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.6410,69.7170,Hydro,500.0,38928341.0,Developing,876000.0
...,...,...,...,...,...,...,...,...,...,...,...
34931,ZMB,Zambia,Ndola,50.0,-12.9667,28.6333,Oil,1160.0,18383956.0,Developing,438000.0
34932,ZMB,Zambia,Nkana,20.0,-12.8167,28.2000,Oil,1160.0,18383956.0,Developing,175200.0
34933,ZMB,Zambia,Victoria Falls,108.0,-17.9167,25.8500,Hydro,1160.0,18383956.0,Developing,946080.0
34934,ZWE,Zimbabwe,Hwange Coal Power Plant Zimbabwe,920.0,-18.3835,26.4700,Coal,1140.0,14862927.0,Developing,8059200.0


In [124]:
# Group the data by the Country Code to get a total MW capacity for all of power plants in the country over the year.

# Reduce the table to just the necessary columns
power_plant_capacity_df_reduced = power_plant_capacity_df[["Country Code", "Power Plant Capacity(MW per Year)"]]

# Group by "Country Code"
power_plant_capacity_df_grouped = power_plant_capacity_df_reduced.groupby(["Country Code"], as_index=False)

# Get the sum of all "Power Plant Capacity(MW per Year)" values for each Country Code
power_plant_capacity_df_grouped = power_plant_capacity_df_grouped.sum()

power_plant_capacity_df_grouped

,Country Code,Power Plant Capacity(MW per Year)
0,AFG,2.632818e+06
1,AGO,9.383537e+06
2,ALB,1.339404e+07
3,ARE,2.656645e+08
4,ARG,2.883186e+08
...,...,...
162,VNM,3.622303e+08
163,YEM,9.154200e+06
164,ZAF,4.417029e+08
165,ZMB,2.355859e+07


In [145]:
# Merge the group by sum of all "Power Plant Capacity(MW per Year)" values for each Country Code dataframe with the power_plant_capacity_df under as a new variable
power_plant_capacity_merged_df = pd.merge(power_plant_capacity_df, power_plant_capacity_df_grouped, how="left", on=["Country Code", "Country Code"])
power_plant_capacity_merged_df

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population,Country Status,Power Plant Capacity(MW per Year)_x,Power Plant Capacity(MW per Year)_y
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.3220,65.1190,Hydro,500.0,38928341.0,Developing,289080.0,2632818.00
1,AFG,Afghanistan,Kandahar DOG,10.0,31.6700,65.7950,Solar,500.0,38928341.0,Developing,87600.0,2632818.00
2,AFG,Afghanistan,Kandahar JOL,10.0,31.6230,65.7920,Solar,500.0,38928341.0,Developing,87600.0,2632818.00
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.5560,69.4787,Hydro,500.0,38928341.0,Developing,578160.0,2632818.00
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.6410,69.7170,Hydro,500.0,38928341.0,Developing,876000.0,2632818.00
...,...,...,...,...,...,...,...,...,...,...,...,...
34931,ZMB,Zambia,Ndola,50.0,-12.9667,28.6333,Oil,1160.0,18383956.0,Developing,438000.0,23558592.12
34932,ZMB,Zambia,Nkana,20.0,-12.8167,28.2000,Oil,1160.0,18383956.0,Developing,175200.0,23558592.12
34933,ZMB,Zambia,Victoria Falls,108.0,-17.9167,25.8500,Hydro,1160.0,18383956.0,Developing,946080.0,23558592.12
34934,ZWE,Zimbabwe,Hwange Coal Power Plant Zimbabwe,920.0,-18.3835,26.4700,Coal,1140.0,14862927.0,Developing,8059200.0,14629200.00


In [150]:
# clean up the power_plant_capacity_merged_df dataframe

power_plant_capacity_merged_df = power_plant_capacity_merged_df[['Country Name', 'Country Code', 'Country Status', 'Population', 'Power Plant Capacity(MW per Year)_y']]
power_plant_capacity_merged_df

,Country Name,Country Code,Country Status,Population,Power Plant Capacity(MW per Year)_y
0,Afghanistan,AFG,Developing,38928341.0,2632818.00
1,Afghanistan,AFG,Developing,38928341.0,2632818.00
2,Afghanistan,AFG,Developing,38928341.0,2632818.00
3,Afghanistan,AFG,Developing,38928341.0,2632818.00
4,Afghanistan,AFG,Developing,38928341.0,2632818.00
...,...,...,...,...,...
34931,Zambia,ZMB,Developing,18383956.0,23558592.12
34932,Zambia,ZMB,Developing,18383956.0,23558592.12
34933,Zambia,ZMB,Developing,18383956.0,23558592.12
34934,Zimbabwe,ZWE,Developing,14862927.0,14629200.00


In [130]:
power_plant_capacity_merged_df.rename(columns={'Power Plant Capacity(MW per Year)_y': 'Countries Combined Power Plant Capacity (MW per Year)'}, inplace=True)

power_plant_capacity_merged_df

C:\Users\carly\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Country Name,Country Code,Country Status,Population,Countries Combined Power Plant Capacity (MW per Year)
0,Afghanistan,AFG,Developing,38928341.0,2632818.00
1,Afghanistan,AFG,Developing,38928341.0,2632818.00
2,Afghanistan,AFG,Developing,38928341.0,2632818.00
3,Afghanistan,AFG,Developing,38928341.0,2632818.00
4,Afghanistan,AFG,Developing,38928341.0,2632818.00
...,...,...,...,...,...
34931,Zambia,ZMB,Developing,18383956.0,23558592.12
34932,Zambia,ZMB,Developing,18383956.0,23558592.12
34933,Zambia,ZMB,Developing,18383956.0,23558592.12
34934,Zimbabwe,ZWE,Developing,14862927.0,14629200.00


In [132]:
# Add a new column to calculate the Countries Combined Power Plant Capacity (MW per Year) divided by the population

power_plant_capacity_merged_df["Countries Combined Power Plant Capacity (MW per Year) Per Capita"] = ((power_plant_capacity_merged_df["Countries Combined Power Plant Capacity (MW per Year)"]) / (power_plant_capacity_merged_df["Population"]))
power_plant_capacity_merged_df

C:\Users\carly\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country Name,Country Code,Country Status,Population,Countries Combined Power Plant Capacity (MW per Year),Countries Combined Power Plant Capacity (MW per Year) Per Capita
0,Afghanistan,AFG,Developing,38928341.0,2632818.00,0.067632
1,Afghanistan,AFG,Developing,38928341.0,2632818.00,0.067632
2,Afghanistan,AFG,Developing,38928341.0,2632818.00,0.067632
3,Afghanistan,AFG,Developing,38928341.0,2632818.00,0.067632
4,Afghanistan,AFG,Developing,38928341.0,2632818.00,0.067632
...,...,...,...,...,...,...
34931,Zambia,ZMB,Developing,18383956.0,23558592.12,1.281476
34932,Zambia,ZMB,Developing,18383956.0,23558592.12,1.281476
34933,Zambia,ZMB,Developing,18383956.0,23558592.12,1.281476
34934,Zimbabwe,ZWE,Developing,14862927.0,14629200.00,0.984274


In [152]:
country_power_capacity_per_capita_df = power_plant_capacity_merged_df.drop_duplicates()
country_power_capacity_per_capita_df = country_power_capacity_per_capita_df.dropna()
country_power_capacity_per_capita_df

,Country Name,Country Code,Country Status,Population,Power Plant Capacity(MW per Year)_y
0,Afghanistan,AFG,Developing,38928341.0,2.632818e+06
9,Albania,ALB,Developing,2837743.0,1.339404e+07
17,Algeria,DZA,Developing,43851043.0,1.390545e+08
76,Angola,AGO,Developing,32866268.0,9.383537e+06
92,Argentina,ARG,Developing,45376763.0,2.883186e+08
...,...,...,...,...,...
34543,Uruguay,URY,Developed,3473727.0,3.687189e+07
34616,Uzbekistan,UZB,Developing,34232050.0,1.107264e+08
34675,Vietnam,VNM,Developing,97338583.0,3.622303e+08
34919,Zambia,ZMB,Developing,18383956.0,2.355859e+07


What can you do with a megawatt-hour of electricity?
(https://www.freeingenergy.com/what-is-a-megawatt-hour-of-electricity-and-what-can-you-do-with-it/)

    Power the average American home for 1.2 months.
    Drive an electric vehicle 3,600 miles.
    Power two 60-watt lightbulbs non-stop for a year.
    Smelt 137 pounds of aluminum.
    Toast 89,000 slices of bread.
    Run an average home pool pump for 5 months.


What can you run on 1 kWh?
(https://www.novec.com/Save/1-kWh.cfm)

    Blend 200 smoothies.
    Microwave 21 frozen burritos.
    Trim 3 miles of weeds.
    Make 60 quarts of ice cream.
    Keep cool with 20 hours of a ceiling fan.
    Binge watch 3 hours of television on a plasma TV or 6 hours on an LCD TV.
    Blow-dry 20 heads of hair.
    Toast 160 slices of bread.


QUICK MATHS: One megawatt (MW) = 1,000 kilowatts 